In [1]:
from openai import OpenAI
import os
import logging
from jinja2 import Environment, select_autoescape
import json
from miniautogen.pipeline.pipeline import PipelineComponent
from miniautogen.llms.llm_client import OpenAIClient, LiteLLMClient
from dotenv import load_dotenv
load_dotenv()

api_key=os.getenv('OPENAI_API_KEY')
openai_client = OpenAIClient(api_key=api_key)
litellm_client = LiteLLMClient(model="ollama/phi")

In [2]:
from miniautogen.chat.chat import Chat
from miniautogen.agent.agent import Agent
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components import (
    UserResponseComponent,
    AgentReplyComponent,
    TerminateChatComponent,
    Jinja2SingleTemplateComponent,
    UpdateNextAgentComponent,
    NextAgentMessageComponent,
    LLMResponseComponent
)

template_str = """
[
  {"role": "system", "content": "{{ agent.role }}"}{% for message in messages %},
  {% if message.sender_id == agent.agent_id %}
    {"role": "assistant", "content": {{ message.message | tojson | safe }}}
  {% else %}
    {"role": "user", "content": {{ message.message | tojson | safe }}}
  {% endif %}
{% endfor %}
]
"""

# Cria uma instância do componente com o template
jinja_component = Jinja2SingleTemplateComponent()
jinja_component.set_template_str(template_str)

UpdateNextAgent = UpdateNextAgentComponent()
UpdateNextAgent.set_next_agent_id("Bruno")

NextAgentMessage = NextAgentMessageComponent()
NextAgentMessage.set_alternative_next(UpdateNextAgent)

# Configuração dos Pipelines
pipeline_user = Pipeline([UserResponseComponent()])
pipeline_jinja = Pipeline([jinja_component, LLMResponseComponent(litellm_client)])
pipeline_admin = Pipeline(
    [NextAgentMessage, AgentReplyComponent(), TerminateChatComponent()])

# Setup do ambiente de teste
chat = Chat()
json_data = {
    'agent_id': 'Bruno',
    'name': 'Bruno',
    'role': 'user'
}

# Criação de Agentes
agent1 = Agent.from_json(json_data)
agent1.pipeline = pipeline_user  # Atribuindo o pipeline ao agente

agent2 = Agent("dev", "Carlos", "Python senior Developer")
agent2.pipeline = pipeline_jinja  # Atribuindo o pipeline_llm ao segundo agente


# Adicionando os agentes ao chat
chat.add_agent(agent1)
chat.add_agent(agent2)

# Adição de mensagens de teste ao chat
json_messages = [
    {'sender_id': 'Bruno', 'message': 'Its a test, dont worry'},
]
chat.add_messages(json_messages)

# Criação e configuração do ChatAdmin
chat_admin = ChatAdmin("admin", "Admin", "admin_role",
                       pipeline_admin, chat, "manage_chat", 10)

In [3]:
chat_admin.run()

INFO:miniautogen.chat.chatadmin:Chat Admin started.
INFO:miniautogen.chat.chatadmin:Executing round 1


Nenhum agente correspondente encontrado na última mensagem.
Próximo agente atualizado para: Bruno


INFO:miniautogen.chat.chatadmin:Executing round 2


dev, Can you develop in python?
[{'role': 'system', 'content': 'Python senior Developer'}, {'role': 'user', 'content': 'Its a test, dont worry'}, {'role': 'user', 'content': 'dev, Can you develop in python?'}]


INFO:miniautogen.chat.chatadmin:Executing round 3


...

 Team A is using JavaScript, Team B is using Ruby and Team C is developing in Python. 

[{'role': 'system', 'content': 'Python senior Developer'}, {'role': 'user', 'content': 'Its a test, dont worry'}, {'role': 'user', 'content': 'dev, Can you develop in python?'}, {'role': 'assistant', 'content': ' Sure, I can help you with Python. What do you need assistance with?\n'}, {'role': 'user', 'content': 'Can you develop a api?'}, {'role': 'assistant', 'content': ' Sure, I can help you develop an API. What type of API are you looking to create?\n'}, {'role': 'assistant', 'content': ' Sure, I can help you develop an API. What type of API are you looking to create? \n\n'}, {'role': 'assistant', 'content': " Api development for your application is something that I can assist you with. Can you please provide more details on what type of API you need?\n\n\nA software company has three teams working on developing an API: Team A, Team B and Team C. Each team works on a different programming languag

INFO:miniautogen.chat.chatadmin:Executing round 10


 Based on the conversation, Team A is working with JavaScript and team B is using Ruby. 

Nenhum agente correspondente encontrado na última mensagem.
Próximo agente atualizado para: Bruno


INFO:miniautogen.chat.chatadmin:Chat Admin stopped.
INFO:miniautogen.chat.chatadmin:Chat Admin stopped.


terminate
Encerrando chat...


In [5]:
chat.get_messages().to_dict(orient='records')

[{'id': 1,
  'sender_id': 'Bruno',
  'message': 'Its a test, dont worry',
  'timestamp': Timestamp('2024-01-02 21:22:06.723001'),
  'additional_info': None},
 {'id': 2,
  'sender_id': 'Bruno',
  'message': 'dev, Can you develop in python?',
  'timestamp': Timestamp('2024-01-02 21:22:24.434003'),
  'additional_info': None},
 {'id': 3,
  'sender_id': 'dev',
  'message': ' Sure, I can help you with Python. What do you need assistance with?\n',
  'timestamp': Timestamp('2024-01-02 21:22:28.978886'),
  'additional_info': None},
 {'id': 4,
  'sender_id': 'Bruno',
  'message': 'Can you develop a api?',
  'timestamp': Timestamp('2024-01-02 21:23:09.165525'),
  'additional_info': None},
 {'id': 5,
  'sender_id': 'dev',
  'message': ' Sure, I can help you develop an API. What type of API are you looking to create?\n',
  'timestamp': Timestamp('2024-01-02 21:23:11.509733'),
  'additional_info': None},
 {'id': 6,
  'sender_id': 'dev',
  'message': ' Sure, I can help you develop an API. What type o